## Install neccessary packages 

In [1]:
!pip install requests

## Fetch and save data from API

### Import neccessary packages

In [2]:
import requests
import sqlite3
from datetime import datetime, timedelta

### Create database schema

In [23]:
conn = sqlite3.connect('mars_photos_stats.db')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS photos_stats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    earth_date TEXT,
    camera_name TEXT,
    photo_count INTEGER,
    rover_status TEXT
)''')

conn.commit()

### Define helper functions for fetch photos from API by specific date

In [10]:
def fetch_photos(earth_date, base_url, rover, api_key):
    url = f"{base_url}/{rover}/photos"
    params = {
        'api_key': api_key,
        'earth_date': earth_date
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()['photos']
    else:
        print(f"Error: {response.status_code} on date {earth_date}")
        return []

### Define main function for retrieving data and save it to DB from API

In [22]:
def process_and_store_data(start_date, end_date, base_url, rover, api_key):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    delta = timedelta(days=1)

    while start <= end:
        earth_date = start.strftime('%Y-%m-%d')
        print(f"Fetching data for {earth_date}")
        photos = fetch_photos(earth_date, base_url, rover, api_key)

        camera_counts = {}
        for photo in photos:
            camera = photo['camera']['name']
            camera_counts[camera] = camera_counts.get(camera, 0) + 1

        rover_status = photos[-1]['rover'].get('status', 'unknown') if photos else 'unknown'

        for camera, count in camera_counts.items():
            c.execute('''INSERT INTO photos_stats (earth_date, camera_name, photo_count, rover_status)
                         VALUES (?, ?, ?, ?)''',
                      (earth_date, camera, count, rover_status))
        
        conn.commit()
        start += delta

### Call main function and specify contstants

In [24]:
API_KEY = 'Token'
ROVER = 'Curiosity'
BASE_URL = 'https://api.nasa.gov/mars-photos/api/v1/rovers'
START_DATE = '2022-01-01'
END_DATE = '2022-01-05'

process_and_store_data(START_DATE, END_DATE, BASE_URL, ROVER, API_KEY)

Fetching data for 2022-01-01
Fetching data for 2022-01-02
Fetching data for 2022-01-03
Fetching data for 2022-01-04
Fetching data for 2022-01-05


### Check data at DB

In [21]:
c.execute('SELECT * FROM photos_stats')
results = c.fetchall()
print(results)
conn.close()

[(1, '2022-01-01', 'FHAZ', 5, 'active'), (2, '2022-01-01', 'RHAZ', 2, 'active'), (3, '2022-01-01', 'MAST', 353, 'active'), (4, '2022-01-01', 'CHEMCAM', 24, 'active'), (5, '2022-01-01', 'MAHLI', 68, 'active'), (6, '2022-01-01', 'MARDI', 2, 'active'), (7, '2022-01-01', 'NAVCAM', 27, 'active'), (8, '2022-01-02', 'FHAZ', 2, 'active'), (9, '2022-01-02', 'RHAZ', 2, 'active'), (10, '2022-01-02', 'MAST', 221, 'active'), (11, '2022-01-02', 'CHEMCAM', 4, 'active'), (12, '2022-01-02', 'MAHLI', 16, 'active'), (13, '2022-01-02', 'MARDI', 2, 'active'), (14, '2022-01-02', 'NAVCAM', 196, 'active'), (15, '2022-01-03', 'CHEMCAM', 3, 'active'), (16, '2022-01-03', 'NAVCAM', 130, 'active'), (17, '2022-01-05', 'FHAZ', 4, 'active'), (18, '2022-01-05', 'RHAZ', 2, 'active'), (19, '2022-01-05', 'MAST', 237, 'active'), (20, '2022-01-05', 'CHEMCAM', 32, 'active'), (21, '2022-01-05', 'MAHLI', 33, 'active'), (22, '2022-01-05', 'MARDI', 2, 'active'), (23, '2022-01-05', 'NAVCAM', 178, 'active')]
